# Initial processing of GMOS-N and GMOS-S flatfield files

## Before running notebook:

- Install <tt>fpack</tt> and <tt>funpack</tt> if not already installed (https://heasarc.gsfc.nasa.gov/fitsio/fpack/); change paths to local installation location in functions <tt>compress_file_fpack</tt> and <tt>decompress_file_fpack</tt> below<br><br>

- Go to https://archive.gemini.edu/searchform; specify UTC Date range of interest, "GMOS-N" or "GMOS-S" for Instrument, "Twilight" as Target Name, "2x2" for Binning, and "Full Frame" for ROI, and specify filter of interest; search for flatfield images; download data<br><br>

- Go to https://archive.gemini.edu/searchform; specify UTC Date of corresponding flatfield data, "GMOS-N" or "GMOS-S" for Instrument, "BIAS" as Obs. Type, "2x2" for Binning, and "Full Frame" for ROI; search for bias frames corresponding to each night of flatfield data; download data<br><br>

- Unpack compressed archives into a new directory, putting raw flatfield data in a subdirectory called "rawfits_twiskyflat" and raw bias data in a subdirectory called "rawfits_bias"<br><br>

- Run <tt>process_flatfield_directory(data_path)</tt> where data_path is location of directory containing "<tt>rawfits_twiskyflat</tt>" and "<tt>rawfits_bias</tt>" subdirectories

In [ ]:
%matplotlib inline
import os, sys
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData
import numpy as np
import glob, subprocess
import datetime


In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    process = subprocess.call(['bzip2','-d',filename])
    return None

def compress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/fpack',filename])
    os.remove(filename)
    return None

def decompress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/funpack',filename])
    os.remove(filename)
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in glob.glob('*.bz2'):
        decompress_file_bzip2(bz2_file)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None

def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    for raw_mef_file in glob.glob('*.fits'):
        fits_date = raw_mef_file[1:9]
    return fits_date

def create_stats_files(fits_date):
    for extid in range(2,10):
        ext_stats = 'n' + fits_date + '.{:02d}.stats'.format(extid+1)
        with open(ext_stats,'w') as of:
            of.write('# Extension {:02d}              NPIX        MEAN     STDDEV         MIN         MAX\n'.format(extid+1))
    return None
        
def compile_stats_files(fits_date,outputfile_path):
    print('Starting image statistics output...')
    output_stats_filename = outputfile_path + 'n' + fits_date + '.stats'
    output_stats_file = open(output_stats_filename,'w')
    for stats_filename in glob.glob('*.??.stats'):
        stats_file = open(stats_filename,'r')
        for line in stats_file:
            output_stats_file.write(line)
        stats_file.close()
        os.remove(stats_filename)
    output_stats_file.close()
    print('>>> Image statistics output complete.')
    return None

def split_extensions():
    # split 12-element GMOS multi-extension FITS files into individual elements
    # return: dimensions of extensions; also writes individual extension files to working directory
    for raw_mef_file in glob.glob('*.fits'):
        with fits.open(raw_mef_file) as hdulist:
            print('Splitting {:s}...'.format(raw_mef_file))
            fits_date = raw_mef_file[1:9]
            fits_id   = raw_mef_file[11:14]
            hdr1 = getheader(raw_mef_file,0)
            for extid in range(2,10):
                extension = hdulist[extid+1].data
                hdr2 = hdulist[extid+1].header
                radecsys = hdr2['RADECSYS']
                hdr2['RADESYSA'] = radecsys
                del hdr2['RADECSYS']
                imstat_npix              = extension.size
                imstat_min,imstat_max    = np.min(extension),np.max(extension)
                imstat_mean,imstat_stdev = np.mean(extension),np.std(extension)
                ext_filename = 'n' + fits_date + '.' + fits_id + '.{:02d}.fits'.format(extid+1)
                outputfilename = 'n' + fits_date + '.{:02d}.stats'.format(extid+1)
                with open(outputfilename,'a') as of:
                    of.write('{:s}   {:>8d}    {:>8.2f}    {:>7.2f}    {:>8.2f}    {:>8.2f}\n'.format(ext_filename,imstat_npix,imstat_mean,imstat_stdev,imstat_min,imstat_max))
                dimensions = extension.shape
                dimension1,dimension2 = dimensions[0],dimensions[1]
                hdr = hdr1 + hdr2
                fits.writeto(ext_filename,extension,hdr)
    return dimension1,dimension2

def overscan_and_trim():
    # Do overscan correction and trim images in current directory
    print('Starting overscan correction and trimming...')
    for fits_file in glob.glob('*.???.??.fits'):
        ot_fits_file = fits_file[0:16] + '.ot.fits'
        fits_data = CCDData.read(fits_file,unit=u.adu)
        file_ext = fits_file[14:16]
        # Overscan correction
        if ((file_ext == '03') or (file_ext=='05') or (file_ext=='07') or (file_ext=='09')):
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[257:288,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        else:
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[1:32,1:2112]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        # Image trimming
        if ((file_ext == '03') or (file_ext=='07')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[1:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '04') or (file_ext == '08')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:282,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        elif ((file_ext == '05') or (file_ext =='09')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[7:256,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        else: # file_ext == '02' or file_ext == '10'
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:288,65:2110]',add_keyword={'trimmed': True, 'calstat': 'OT'})
        ot_fits_data.write(ot_fits_file)
        os.remove(fits_file)
    print('>>> Overscan correction and trimming complete.')
    return None

def bias_median_combine(dateid):
    print('Starting median combination of bias frames...')
    for extid in range(2,10):
        bias_list = []
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        output_filename = 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        for fits_file in glob.glob(ext_file_format):
            fits_data = CCDData.read(fits_file)
            bias_list.append(fits_data)
            os.remove(fits_file)
        master_bias = cp.combine(bias_list,method='median')
        master_bias.write(output_filename)
    print('>>> Median combination of bias frames complete.')
    return None

def bias_correct(dateid,cwd_raw_bias):
    print('Starting bias subtraction...')
    for extid in range(2,10):
        bias_filename = cwd_raw_bias + 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
        ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
        for fits_file in glob.glob(ext_file_format):
            fits_data = CCDData.read(fits_file)
            bias_data = CCDData.read(bias_filename)
            fits_date_imageid = fits_file[0:13]
            output_filename = fits_date_imageid + '.{:02d}.otz.fits'.format(extid+1)
            bias_corrected_data = cp.subtract_bias(fits_data,bias_data,add_keyword={'zerocorr': True, 'calstat': 'OTZ'})
            bias_corrected_data.write(output_filename)
            os.remove(fits_file)
    print('>>> Bias subtraction complete.')
    return None
    
def concatenate_gmos_amps():
    print('>>> Starting adjacent amp area concatenation...')
    for fits_file in glob.glob('*.03.otz.fits'):
        file_prefix = fits_file[0:13]
        ext1_filename = file_prefix + '.03.otz.fits'
        ext2_filename = file_prefix + '.04.otz.fits'
        hdr = getheader(ext1_filename,0)
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2:
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip1.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        os.remove(ext1_filename)
        os.remove(ext2_filename)

        ext1_filename = file_prefix + '.05.otz.fits'
        ext2_filename = file_prefix + '.06.otz.fits'
        ext3_filename = file_prefix + '.07.otz.fits'
        ext4_filename = file_prefix + '.08.otz.fits'
        hdr = getheader(ext1_filename,0)
        
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2, fits.open(ext3_filename) as hdul3, fits.open(ext4_filename) as hdul4:
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
            ext3_data = hdul3[0].data        
            ext4_data = hdul4[0].data
            
        chip1_data = np.hstack((ext1_data,ext2_data,ext3_data,ext4_data))
        output_filename = file_prefix + '.chip2.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        
        os.remove(ext1_filename)
        os.remove(ext2_filename)
        os.remove(ext3_filename)
        os.remove(ext4_filename)

        ext1_filename = file_prefix + '.09.otz.fits'
        ext2_filename = file_prefix + '.10.otz.fits'
        hdr = getheader(ext1_filename,0)
        
        with fits.open(ext1_filename) as hdul1, fits.open(ext2_filename) as hdul2:
            ext1_data = hdul1[0].data
            ext2_data = hdul2[0].data
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip3.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        os.remove(ext1_filename)
        os.remove(ext2_filename)
    print('>>> Adjacent amp area concatenation complete.')
    return None


def process_flatfield_files(path_rawfits,path_rawbias,path_procfits):
    
    print('\n')
    create_directory(path_procfits)
    
    # Process raw science fits files
    print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawfits)
    decompress_directory_bzip2(path_rawfits)      #decompress downloaded GMOS files
    decompress_directory_fpack(path_rawfits)      #decompress previously fpacked GMOS files
    date_id = get_date_id()
    create_stats_files(date_id)                   #create files to record image statistics of bias files
    dim1,dim2 = split_extensions()                #split science MEFs into individual FITS files
    compile_stats_files(date_id,path_procfits)    #collect image statistics of science files together and delete individual files

    # decompress raw bias fits files
    print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawbias)
    decompress_directory_bzip2(path_rawbias)      #decompress downloaded GMOS files
    decompress_directory_fpack(path_rawbias)      #decompress previously fpacked GMOS files
    create_stats_files(date_id)                   #create files to record image statistics of bias files
    split_extensions()                            #split bias MEFs into individual FITS files
    compile_stats_files(date_id+'.bias',path_procfits)          #collect image statistics of bias files together and delete individual files

    # perform overscan corrections and trim data for bias frames
    os.chdir(path_rawbias)
    overscan_and_trim()
    bias_median_combine(date_id)

    # perform overscan corrections and trim data for science frames
    os.chdir(path_rawfits)
    overscan_and_trim()
    bias_correct(date_id,path_rawbias)

    # join science frames
    os.chdir(path_rawfits)
    concatenate_gmos_amps()
    
    # move processed files to processed directory
    os.chdir(path_rawfits)
    for fits_file in glob.glob('*.chip?.otz.fits'):
        os.rename(fits_file,path_procfits+fits_file)
    os.chdir(path_rawbias)
    for fits_file in glob.glob('*.bias.??.fits'):
        os.rename(fits_file,path_procfits+fits_file)

    compress_directory_fpack(path_rawfits)
    compress_directory_fpack(path_rawbias)
    compress_directory_fpack(path_procfits)
        
    return None


In [ ]:
def process_flatfield_directory(data_path):
    os.chdir(data_path)
    path_rawfits  = data_path + 'rawfits_twiskyflat/'
    path_rawbias  = data_path + 'rawfits_bias/'
    path_procfits = data_path + 'procfits_twiskyflat/'
    
    if os.path.exists(path_procfits):    
        print('{:s} - Data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    elif not os.path.exists(path_rawfits):
        print('{:s} - Raw flatfield directory ({:s}) not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    elif not os.path.exists(path_rawbias):
        print('{:s} - Bias directory ({:s}) not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    elif len(os.listdir(path_rawfits)) == 0:
        print('{:s} - Raw flatfield directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    elif len(os.listdir(path_rawbias)) == 0:
        print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    else:
        process_flatfield_files(path_rawfits,path_rawbias,path_procfits)

    return None
    

In [ ]:
process_flatfield_directory(data_path)
